## 3-5-benzenetricarboxylicacid

## You have to visualize the structures after relaxation!!

In [1]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_1-3-5-benzenetricarboxylicacid_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [2]:
xyz_path = 'savs_xyz_files/ni_oct_2_1-3-5-benzenetricarboxylicacid_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

$$\Delta E_{ads} = (E_{complex+H}+E_{separate ligand}) - E_{complex} - E_H$$
### We need to add the energy of the separate ligand because in the hydride complex, we remove a ligand to add the hydrogen.

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "OC(=O)c1cc(cc(c1)C(O)=O)C(O)=O"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OCOC7O2CO2H6', pbc=False)


## Compute the Energy of the Complex

In [4]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

W1123 20:37:15.912000 83098 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


      Step     Time          Energy          fmax
FIRE:    0 20:37:30  -106136.578863       18.065716
FIRE:    1 20:37:31  -106141.525669       10.055765
FIRE:    2 20:37:31  -106143.554307        4.003895
FIRE:    3 20:37:32  -106143.774231        2.874483
FIRE:    4 20:37:33  -106143.918839        2.543932
FIRE:    5 20:37:34  -106144.123161        2.080614
FIRE:    6 20:37:35  -106144.280322        1.462455
FIRE:    7 20:37:36  -106144.348995        1.673779
FIRE:    8 20:37:37  -106144.382835        2.188252
FIRE:    9 20:37:38  -106144.398285        2.060860
FIRE:   10 20:37:39  -106144.425461        1.824489
FIRE:   11 20:37:40  -106144.458153        1.513819
FIRE:   12 20:37:41  -106144.489750        1.177893
FIRE:   13 20:37:42  -106144.515309        0.882920
FIRE:   14 20:37:43  -106144.533189        0.712052
FIRE:   15 20:37:44  -106144.545522        0.746071
FIRE:   16 20:37:45  -106144.557657        0.879514
FIRE:   17 20:37:45  -106144.574261        0.923609
FIRE:   18 20:

In [5]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 20:42:33   -84466.873067       23.341059
FIRE:    1 20:42:33   -84471.859528        9.053540
FIRE:    2 20:42:34   -84473.302407        1.965612
FIRE:    3 20:42:35   -84472.955491        5.361577
FIRE:    4 20:42:35   -84473.156355        4.777818
FIRE:    5 20:42:36   -84473.437434        3.677852
FIRE:    6 20:42:37   -84473.651702        2.183313
FIRE:    7 20:42:37   -84473.750812        2.169060
FIRE:    8 20:42:38   -84473.798525        2.506888
FIRE:    9 20:42:40   -84473.814576        2.343838
FIRE:   10 20:42:40   -84473.842692        2.040604
FIRE:   11 20:42:41   -84473.876341        1.639575
FIRE:   12 20:42:42   -84473.908760        1.198444
FIRE:   13 20:42:42   -84473.935274        0.903771
FIRE:   14 20:42:43   -84473.954948        0.873208
FIRE:   15 20:42:44   -84473.970614        1.120837
FIRE:   16 20:42:45   -84473.988454        1.305468
FIRE:   17 20:42:46   -84474.013402        1.336673
FIRE:   18 20:

In [6]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 20:43:26   -21715.577026        5.688187
FIRE:    1 20:43:27   -21716.279486        2.535448
FIRE:    2 20:43:28   -21716.377765        1.735004
FIRE:    3 20:43:28   -21716.463476        1.542774
FIRE:    4 20:43:28   -21716.505577        1.570130
FIRE:    5 20:43:29   -21716.568679        1.826477
FIRE:    6 20:43:30   -21716.674257        1.299854
FIRE:    7 20:43:30   -21716.769791        0.799862
FIRE:    8 20:43:31   -21716.819934        1.594799
FIRE:    9 20:43:31   -21716.884991        1.836215
FIRE:   10 20:43:31   -21716.985876        0.910856
FIRE:   11 20:43:32   -21717.031606        1.253131
FIRE:   12 20:43:32   -21717.039445        1.090762
FIRE:   13 20:43:33   -21717.050946        0.786540
FIRE:   14 20:43:33   -21717.060298        0.388737
FIRE:   15 20:43:34   -21717.064230        0.571339
FIRE:   16 20:43:34   -21717.065224        0.829019
FIRE:   17 20:43:34   -21717.066006        0.800051
FIRE:   18 20:

## Compute the energy of the Hydrogen atom

In [7]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 20:43:44      -31.589716        0.047595
FIRE:    1 20:43:44      -31.589744        0.012354
FIRE:    2 20:43:45      -31.589733        0.031731
FIRE:    3 20:43:45      -31.589737        0.025917
FIRE:    4 20:43:45      -31.589743        0.015339
FIRE:    5 20:43:45      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [8]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand - E_H) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.363 eV


## 5,7dichloro8hydroxyquinoline

In [21]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_57dichloro8hydroxyquinoline_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [27]:
xyz_path = 'savs_xyz_files/ni_oct_2_57dichloro8hydroxyquinoline_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [28]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Oc1c(Cl)cc(Cl)c2cccnc12"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OC2ClC2ClC4NCH5', pbc=False)


In [29]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:00:54  -154985.657393        2.346741
FIRE:    1 21:00:54  -154986.047949        1.052981
FIRE:    2 21:00:55  -154985.986376        1.984502
FIRE:    3 21:00:56  -154986.079707        1.453568
FIRE:    4 21:00:56  -154986.179541        0.712975
FIRE:    5 21:00:57  -154986.205667        0.627874
FIRE:    6 21:00:58  -154986.208243        0.594280
FIRE:    7 21:00:59  -154986.212926        0.529533
FIRE:    8 21:01:00  -154986.218918        0.438400
FIRE:    9 21:01:02  -154986.225326        0.376536
FIRE:   10 21:01:03  -154986.231447        0.363145
FIRE:   11 21:01:04  -154986.237036        0.347231
FIRE:   12 21:01:06  -154986.242343        0.329062
FIRE:   13 21:01:07  -154986.248533        0.324091
FIRE:   14 21:01:08  -154986.256377        0.310034
FIRE:   15 21:01:10  -154986.266308        0.289333
FIRE:   16 21:01:13  -154986.277798        0.282643
FIRE:   17 21:01:18  -154986.289324        0.273241
FIRE:   18 21:

## remove the hydrogen from the oxygen attached 

In [30]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:01:54  -117035.005892        3.806970
FIRE:    1 21:01:54  -117035.489421        1.916155
FIRE:    2 21:01:55  -117035.547104        2.235795
FIRE:    3 21:01:55  -117035.636309        1.570194
FIRE:    4 21:01:56  -117035.732194        0.763627
FIRE:    5 21:01:58  -117035.763863        0.713423
FIRE:    6 21:01:59  -117035.767361        0.711407
FIRE:    7 21:02:00  -117035.773855        0.707362
FIRE:    8 21:02:01  -117035.782459        0.701247
FIRE:    9 21:02:02  -117035.792227        0.692991
FIRE:   10 21:02:02  -117035.802379        0.682498
FIRE:   11 21:02:03  -117035.812608        0.669720
FIRE:   12 21:02:04  -117035.823156        0.654730
FIRE:   13 21:02:06  -117035.835760        0.635843
FIRE:   14 21:02:08  -117035.851406        0.612630
FIRE:   15 21:02:09  -117035.870657        0.584759
FIRE:   16 21:02:10  -117035.892915        0.552088
FIRE:   17 21:02:11  -117035.916445        0.514928
FIRE:   18 21:

In [31]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 21:03:07   -37996.368095        3.890690
FIRE:    1 21:03:07   -37996.682535        1.679395
FIRE:    2 21:03:07   -37996.615606        2.656431
FIRE:    3 21:03:08   -37996.689228        2.020107
FIRE:    4 21:03:08   -37996.768737        1.316344
FIRE:    5 21:03:08   -37996.794396        1.051413
FIRE:    6 21:03:09   -37996.796735        1.001909
FIRE:    7 21:03:09   -37996.801055        0.905497
FIRE:    8 21:03:10   -37996.806737        0.869032
FIRE:    9 21:03:10   -37996.813071        0.854170
FIRE:   10 21:03:10   -37996.819471        0.836125
FIRE:   11 21:03:11   -37996.825664        0.815225
FIRE:   12 21:03:11   -37996.831770        0.791931
FIRE:   13 21:03:11   -37996.838874        0.763961
FIRE:   14 21:03:12   -37996.847655        0.730841
FIRE:   15 21:03:12   -37996.858579        0.691621
FIRE:   16 21:03:13   -37996.871317        0.644401
FIRE:   17 21:03:13   -37996.884542        0.585645
FIRE:   18 21:

In [32]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 21:03:20      -31.589716        0.047596
FIRE:    1 21:03:20      -31.589744        0.012354
FIRE:    2 21:03:20      -31.589733        0.031731
FIRE:    3 21:03:20      -31.589737        0.025917
FIRE:    4 21:03:21      -31.589743        0.015339
FIRE:    5 21:03:21      -31.589747        0.001920
Isolated H atom energy: -15.795 eV


In [35]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand - E_H) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.409 eV


## cyclohexanecarboxylicacid

In [55]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_cyclohexanecarboxylicacid_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [56]:
xyz_path = 'savs_xyz_files/ni_oct_2_cyclohexanecarboxylicacid_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [57]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "OC(=O)C1CCCCC1"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OCOC6H12', pbc=False)


In [58]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:44:20   -75634.181706       18.030838
FIRE:    1 21:44:21   -75639.140186        9.986846
FIRE:    2 21:44:21   -75641.059869        3.994994
FIRE:    3 21:44:22   -75641.160574        3.145979
FIRE:    4 21:44:23   -75641.326180        2.780010
FIRE:    5 21:44:24   -75641.560472        2.163354
FIRE:    6 21:44:26   -75641.731900        1.560864
FIRE:    7 21:44:27   -75641.785171        2.091099
FIRE:    8 21:44:29   -75641.797755        1.967767
FIRE:    9 21:44:31   -75641.820194        1.745232
FIRE:   10 21:44:32   -75641.847873        1.435562
FIRE:   11 21:44:34   -75641.875706        1.069392
FIRE:   12 21:44:36   -75641.899834        0.777710
FIRE:   13 21:44:37   -75641.918954        0.814676
FIRE:   14 21:44:39   -75641.934798        0.885266
FIRE:   15 21:44:41   -75641.952575        0.948065
FIRE:   16 21:44:42   -75641.976681        0.932090
FIRE:   17 21:44:44   -75642.010289        0.854143
FIRE:   18 21:

In [59]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:46:17   -64132.643514       23.621708
FIRE:    1 21:46:19   -64137.626756        9.320072
FIRE:    2 21:46:20   -64138.965726        2.542777
FIRE:    3 21:46:21   -64138.477781        5.706855
FIRE:    4 21:46:22   -64138.705998        5.029996
FIRE:    5 21:46:22   -64139.025796        3.772164
FIRE:    6 21:46:23   -64139.260591        2.090457
FIRE:    7 21:46:24   -64139.349046        2.391569
FIRE:    8 21:46:25   -64139.365057        2.217004
FIRE:    9 21:46:26   -64139.392913        1.892835
FIRE:   10 21:46:27   -64139.425868        1.464673
FIRE:   11 21:46:28   -64139.457072        0.993374
FIRE:   12 21:46:29   -64139.482040        0.927149
FIRE:   13 21:46:30   -64139.500379        0.909464
FIRE:   14 21:46:31   -64139.515556        1.207185
FIRE:   15 21:46:32   -64139.534334        1.459268
FIRE:   16 21:46:33   -64139.562235        1.539960
FIRE:   17 21:46:34   -64139.602486        1.391254
FIRE:   18 21:

In [60]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 21:47:16   -11549.009501        5.280601
FIRE:    1 21:47:17   -11549.254228        2.474282
FIRE:    2 21:47:17   -11549.288058        1.610346
FIRE:    3 21:47:18   -11549.322268        1.452121
FIRE:    4 21:47:18   -11549.346343        1.441758
FIRE:    5 21:47:19   -11549.378194        1.656641
FIRE:    6 21:47:19   -11549.422878        1.095915
FIRE:    7 21:47:20   -11549.462415        0.697082
FIRE:    8 21:47:20   -11549.485103        1.323008
FIRE:    9 21:47:21   -11549.509021        1.443559
FIRE:   10 21:47:21   -11549.535154        0.398810
FIRE:   11 21:47:22   -11549.538155        1.141204
FIRE:   12 21:47:22   -11549.541129        0.974886
FIRE:   13 21:47:23   -11549.545550        0.663777
FIRE:   14 21:47:23   -11549.549292        0.259644
FIRE:   15 21:47:23   -11549.551135        0.443560
FIRE:   16 21:47:24   -11549.551912        0.754123
FIRE:   17 21:47:24   -11549.553375        0.861486
FIRE:   18 21:

In [61]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 21:47:35      -31.589716        0.047595
FIRE:    1 21:47:36      -31.589744        0.012354
FIRE:    2 21:47:36      -31.589733        0.031731
FIRE:    3 21:47:36      -31.589737        0.025917
FIRE:    4 21:47:36      -31.589743        0.015339
FIRE:    5 21:47:37      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [63]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand - E_H) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.723 eV


## 2-mercaptopyridine

In [64]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/2mercaptopyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [65]:
xyz_path = 'complexes/xyz/ni_oct_2_2mercaptopyridine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [66]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "S=C1NC=CC=C1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='SCNC4H5', pbc=False)


## Compared to the ligand in the complex, this ligand has an extra hydrogen attached to the Nitrogen

In [67]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:50:01   -93767.719407        4.768582
FIRE:    1 21:50:02   -93768.915707        2.202767
FIRE:    2 21:50:02   -93769.000154        3.415560
FIRE:    3 21:50:03   -93769.174726        2.487954
FIRE:    4 21:50:03   -93769.367343        1.075715
FIRE:    5 21:50:04   -93769.441146        0.970554
FIRE:    6 21:50:04   -93769.445536        0.934019
FIRE:    7 21:50:05   -93769.453618        0.863256
FIRE:    8 21:50:06   -93769.464188        0.762943
FIRE:    9 21:50:07   -93769.475913        0.640480
FIRE:   10 21:50:08   -93769.487689        0.509099
FIRE:   11 21:50:09   -93769.499024        0.485283
FIRE:   12 21:50:10   -93769.510127        0.538780
FIRE:   13 21:50:11   -93769.522890        0.631177
FIRE:   14 21:50:12   -93769.538416        0.664770
FIRE:   15 21:50:12   -93769.557376        0.618627
FIRE:   16 21:50:13   -93769.579171        0.483622
FIRE:   17 21:50:14   -93769.601728        0.317501
FIRE:   18 21:

In [68]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:50:35   -76223.436266        5.491088
FIRE:    1 21:50:36   -76224.550947        3.115045
FIRE:    2 21:50:37   -76224.671221        3.103370
FIRE:    3 21:50:38   -76224.793823        2.245922
FIRE:    4 21:50:38   -76224.931962        0.929209
FIRE:    5 21:50:39   -76224.986763        0.625549
FIRE:    6 21:50:40   -76224.989105        0.595484
FIRE:    7 21:50:40   -76224.993438        0.593000
FIRE:    8 21:50:41   -76224.999160        0.589301
FIRE:    9 21:50:41   -76225.005588        0.584407
FIRE:   10 21:50:42   -76225.012185        0.578341
FIRE:   11 21:50:42   -76225.018719        0.571147
FIRE:   12 21:50:43   -76225.025310        0.562921
FIRE:   13 21:50:43   -76225.033074        0.552782
FIRE:   14 21:50:44   -76225.042676        0.540478
FIRE:   15 21:50:45   -76225.054595        0.525657
FIRE:   16 21:50:45   -76225.068651        0.507765
FIRE:   17 21:50:46   -76225.083867        0.485923
FIRE:   18 21:

In [69]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")
view(ligand_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 21:50:58   -17591.029900        3.101195
FIRE:    1 21:50:58   -17591.197802        0.951425
FIRE:    2 21:50:58   -17591.147887        2.909638
FIRE:    3 21:50:59   -17591.202806        1.954076
FIRE:    4 21:50:59   -17591.250252        0.693542
FIRE:    5 21:51:00   -17591.248242        1.056051
FIRE:    6 21:51:00   -17591.250764        0.978023
FIRE:    7 21:51:00   -17591.255171        0.827324
FIRE:    8 21:51:01   -17591.260375        0.614957
FIRE:    9 21:51:01   -17591.265164        0.466938
FIRE:   10 21:51:02   -17591.268612        0.415434
FIRE:   11 21:51:02   -17591.270509        0.369015
FIRE:   12 21:51:02   -17591.271523        0.431653
FIRE:   13 21:51:03   -17591.272828        0.550848
FIRE:   14 21:51:03   -17591.275480        0.587667
FIRE:   15 21:51:04   -17591.279974        0.502220
FIRE:   16 21:51:04   -17591.285555        0.307295
FIRE:   17 21:51:04   -17591.290165        0.234497
FIRE:   18 21:

In [70]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 21:51:11      -31.589716        0.047595
FIRE:    1 21:51:11      -31.589744        0.012354
FIRE:    2 21:51:11      -31.589733        0.031731
FIRE:    3 21:51:12      -31.589737        0.025917
FIRE:    4 21:51:12      -31.589743        0.015339
FIRE:    5 21:51:12      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [71]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand - E_H) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.682 eV


In [ ]:
{
 "2-mercaptopyridine":0.682,
}